In [26]:
import pandas as pd
import numpy as np
df = pd.read_csv('data/kc_house_data.csv')

In [27]:
df.head() #original dataframe

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [28]:
#To get edited dataframe, just run this cell first
def make_month(string):
    if string[:2] == '1/':
        return 'Jan'
    if string[:2] == '2/':
        return 'Feb'
    if string[:2] == '3/':
        return 'Mar'
    if string[:2] == '4/':
        return 'Apr'
    if string[:2] == '5/':
        return 'May'
    if string[:2] == '6/':
        return 'Jun'
    if string[:2] == '7/':
        return 'Jul'
    if string[:2] == '8/':
        return 'Aug'
    if string[:2] == '9/':
        return 'Sep'
    if string[:2] == '10':
        return 'Oct'
    if string[:2] == '11':
        return 'Nov'
    if string[:2] == '12':
        return 'Dec'

In [29]:
#Then run this cell
df_alt = df.drop(['id', 'date', 'yr_built', 'yr_renovated', 'lat', 'long'], axis = 1)
month_list = []
sqft_basement_list = []
yr_renovated = []
for date in df['date'].values:
    month_list.append(make_month(date))
for num in df['sqft_basement'].values:
    if num == '?':
        sqft_basement_list.append(np.nan)
    else:
        sqft_basement_list.append(float(num))
for data in df['yr_renovated'].values:
    if data == 0:
        yr_renovated.append(0)
    elif data == np.nan:
        yr_renovated.append(np.nan)
    else:
        yr_renovated.append(1)
        
df_alt['month'] = month_list
df_alt['sqft_basement'] = sqft_basement_list
df_alt['renovated'] = yr_renovated
df_alt['zipcode'] = df_alt['zipcode'].apply(str)
df_alt['age_of_house'] = 2021 - df['yr_built']
df_alt['total_sq_ft'] = df['sqft_above'] + df_alt['sqft_basement']
df_alt.head() #df_alt is the new dataframe


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,zipcode,sqft_living15,sqft_lot15,month,renovated,age_of_house,total_sq_ft
0,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,3,7,1180,0.0,98178,1340,5650,Oct,0,66,1180.0
1,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,3,7,2170,400.0,98125,1690,7639,Dec,1,70,2570.0
2,180000.0,2,1.00,770,10000,1.0,0.0,0.0,3,6,770,0.0,98028,2720,8062,Feb,1,88,770.0
3,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,5,7,1050,910.0,98136,1360,5000,Dec,0,56,1960.0
4,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,3,8,1680,0.0,98074,1800,7503,Feb,0,34,1680.0


In [25]:
# Here is my logic for all the edits I made but of course, feel free to make your own edits too

## dropped'id' because it's useless lol
## dropped 'date' because its too specific and replaced with a column that shows month categorically
## dropped'yr_built' because it doesn't tell us much BUT I also replaced with this column with 
# age_of_house which shows how old the home is in years
## dropped'yr_renovated' and made categorical (renovated) as to whether it has been renovated or not
## dropped 'lat', feels useless
## dropped 'long', feels useless
## made zipcode into a string to make categorical
## made sqft_basement into floats (it was in string format)
## created a total_sq_ft column combining basement sq fit with the rest of the house